In [1]:
using Oceananigans

┌ Warning: The NVIDIA driver on this system only supports up to CUDA 11.0.0.
│ For performance reasons, it is recommended to upgrade to a driver that supports CUDA 11.2 or higher.
└ @ CUDA C:\Users\shiva\.julia\packages\CUDA\DL5Zo\src\initialization.jl:42


In [2]:
grid = RectilinearGrid(size=(1000,500),extent=(2π,2π),topology=(Periodic,Periodic,Flat))

model = NonhydrostaticModel(timestepper = :RungeKutta3,
                              advection = UpwindBiasedFifthOrder(),
                                   grid = grid,
                               buoyancy = nothing,
                                tracers = nothing,
                                closure = IsotropicDiffusivity(ν=1e-9)
                           )

┌ Warning: Inflating model grid halo size to (3, 3, 0) and recreating grid. The model grid will be different from the input grid. To avoid this warning, pass halo=(3, 3, 0) when constructing the grid.
└ @ Oceananigans.Models.NonhydrostaticModels C:\Users\shiva\.julia\packages\Oceananigans\KUskb\src\Models\NonhydrostaticModels\nonhydrostatic_model.jl:130


NonhydrostaticModel{CPU, Float64}(time = 0 seconds, iteration = 0) 
├── grid: RectilinearGrid{Float64, Periodic, Periodic, Flat}(Nx=1000, Ny=500, Nz=1)
├── tracers: ()
├── closure: IsotropicDiffusivity{Oceananigans.TurbulenceClosures.ExplicitTimeDiscretization, Float64, NamedTuple{(), Tuple{}}}
├── buoyancy: Nothing
└── coriolis: Nothing

In [3]:
using Statistics

u,v,w=model.velocities
ui=rand(size(u)...)
vi=rand(size(v)...)
ui .-= mean(ui)
vi .-= mean(vi)

set!(model, u=ui, v=vi)

In [4]:
u,v,w=model.velocities

(u = Field located at (Face, Center, Center)
├── data: OffsetArrays.OffsetArray{Float64, 3, Array{Float64, 3}}, size: (1000, 500, 1)
├── grid: RectilinearGrid{Float64, Periodic, Periodic, Flat}(Nx=1000, Ny=500, Nz=1)
└── boundary conditions: west=Periodic, east=Periodic, south=Periodic, north=Periodic, bottom=Nothing, top=Nothing, immersed=ZeroFlux, v = Field located at (Center, Face, Center)
├── data: OffsetArrays.OffsetArray{Float64, 3, Array{Float64, 3}}, size: (1000, 500, 1)
├── grid: RectilinearGrid{Float64, Periodic, Periodic, Flat}(Nx=1000, Ny=500, Nz=1)
└── boundary conditions: west=Periodic, east=Periodic, south=Periodic, north=Periodic, bottom=Nothing, top=Nothing, immersed=ZeroFlux, w = Field located at (Center, Center, Face)
├── data: OffsetArrays.OffsetArray{Float64, 3, Array{Float64, 3}}, size: (1000, 500, 1)
├── grid: RectilinearGrid{Float64, Periodic, Periodic, Flat}(Nx=1000, Ny=500, Nz=1)
└── boundary conditions: west=Periodic, east=Periodic, south=Periodic, north=Peri

In [5]:
ω = ∂x(v) - ∂y(u)

ω_field = Field(ω)

Field located at (Face, Face, Center) computed from BinaryOperation at (Face, Face, Center)
├── data: OffsetArrays.OffsetArray{Float64, 3, Array{Float64, 3}}, size: (1000, 500, 1)
├── grid: RectilinearGrid{Float64, Periodic, Periodic, Flat}(Nx=1000, Ny=500, Nz=1)
├── operand: BinaryOperation at (Face, Face, Center)
└── status: time=0.0

In [6]:
s=sqrt(u^2+v^2)
s_field=Field(s)

Field located at (Face, Center, Center) computed from UnaryOperation at (Face, Center, Center)
├── data: OffsetArrays.OffsetArray{Float64, 3, Array{Float64, 3}}, size: (1000, 500, 1)
├── grid: RectilinearGrid{Float64, Periodic, Periodic, Flat}(Nx=1000, Ny=500, Nz=1)
├── operand: UnaryOperation at (Face, Center, Center)
└── status: time=0.0

In [7]:
simulation = Simulation(model, Δt=0.02, stop_time=50)

Simulation{typename(NonhydrostaticModel){typename(CPU), Float64}}
├── Model clock: time = 0 seconds, iteration = 0
├── Next time step: 20 ms
├── Elapsed wall time: 0 seconds
├── Stop time: 50 seconds
├── Stop iteration : Inf
├── Wall time limit: Inf
├── Callbacks: typename(OrderedCollections.OrderedDict) with 4 entries:
│   ├── stop_time_exceeded => typename(Callback)
│   ├── stop_iteration_exceeded => typename(Callback)
│   ├── wall_time_limit_exceeded => typename(Callback)
│   └── nan_checker => typename(Callback)
├── Output writers: typename(OrderedCollections.OrderedDict) with no entries
└── Diagnostics: typename(OrderedCollections.OrderedDict) with no entries

In [8]:
progress(sim) = @info "Iteration: $(iteration(sim)), time: $(time(sim))"

simulation.callbacks[:progress] = Callback(progress, IterationInterval(100))

Callback{typeof(progress), IterationInterval}(progress, IterationInterval(100))

In [9]:
simulation.output_writers[:fields] = JLD2OutputWriter(model, (ω=ω_field, s=s_field),
                                                      schedule = TimeInterval(2),
                                                      prefix = "two_dimensional_turbulence2",
                                                      force = true)

JLD2OutputWriter scheduled on TimeInterval(2 seconds):
├── filepath: .\two_dimensional_turbulence2.jld2
├── 2 outputs: (:ω, :s)
├── field slicer: FieldSlicer(:, :, :, with_halos=false)
├── array type: Array{Float32}
├── including: [:grid, :coriolis, :buoyancy, :closure]
└── max filesize: Inf YiB

In [10]:
run!(simulation)

┌ Info: Initializing simulation...
└ @ Oceananigans.Simulations C:\Users\shiva\.julia\packages\Oceananigans\KUskb\src\Simulations\run.jl:168
┌ Info: Iteration: 0, time: 0.0
└ @ Main In[8]:1
┌ Info:     ... simulation initialization complete (17.800 seconds)
└ @ Oceananigans.Simulations C:\Users\shiva\.julia\packages\Oceananigans\KUskb\src\Simulations\run.jl:190
┌ Info: Executing initial time step...
└ @ Oceananigans.Simulations C:\Users\shiva\.julia\packages\Oceananigans\KUskb\src\Simulations\run.jl:113
┌ Info:     ... initial time step complete (41.687 seconds).
└ @ Oceananigans.Simulations C:\Users\shiva\.julia\packages\Oceananigans\KUskb\src\Simulations\run.jl:120
┌ Info: Iteration: 100, time: 1.9999999999999925
└ @ Main In[8]:1
┌ Info: Iteration: 200, time: 3.9800000000000457
└ @ Main In[8]:1
┌ Info: Iteration: 300, time: 5.979999999999958
└ @ Main In[8]:1
┌ Info: Iteration: 400, time: 7.959999999999958
└ @ Main In[8]:1
┌ Info: Iteration: 500, time: 9.939999999999959
└ @ Main In[8]

In [11]:
using JLD2

file=jldopen(simulation.output_writers[:fields].filepath)
iterations = parse.(Int,keys(file["timeseries/t"]))

26-element Vector{Int64}:
    0
  101
  201
  302
  403
  504
  605
  706
  807
  908
 1009
 1110
 1211
 1312
 1413
 1514
 1615
 1715
 1815
 1915
 2015
 2115
 2215
 2315
 2415
 2515

In [12]:
xω, yω, zω = nodes(ω_field)
xs, ys, zs = nodes(s_field)

([4.142623226210386e-20, 0.006283185307179587, 0.012566370614359173, 0.01884955592153876, 0.025132741228718346, 0.031415926535897934, 0.03769911184307752, 0.0439822971502571, 0.05026548245743669, 0.05654866776461628  …  6.220353454107791, 6.22663663941497, 6.23291982472215, 6.239203010029329, 6.245486195336508, 6.2517693806436885, 6.258052565950868, 6.264335751258048, 6.270618936565227, 6.276902121872407], [0.006283185307179582, 0.018849555921538756, 0.03141592653589793, 0.0439822971502571, 0.05654866776461627, 0.06911503837897544, 0.08168140899333462, 0.09424777960769379, 0.10681415022205296, 0.11938052083641214  …  6.1638047863431735, 6.176371156957533, 6.188937527571892, 6.201503898186251, 6.214070268800611, 6.22663663941497, 6.239203010029329, 6.2517693806436885, 6.264335751258047, 6.276902121872406], StepRangeLen(1.0, 0.0, 1))

In [13]:
using Plots

@info "Making a neat movie of vorticity and speed..."

anim = @animate for (i, iteration) in enumerate(iterations)

    @info "Plotting frame $i from iteration $iteration..."

    t = file["timeseries/t/$iteration"]
    ω_snapshot = file["timeseries/ω/$iteration"][:, :, 1]
    s_snapshot = file["timeseries/s/$iteration"][:, :, 1]

    ω_lim = 2.0
    ω_levels = range(-ω_lim, stop=ω_lim, length=20)

    s_lim = 0.2
    s_levels = range(0, stop=s_lim, length=20)

    kwargs = (xlabel="x", ylabel="y", aspectratio=1, linewidth=0, colorbar=true,
              xlims=(0, model.grid.Lx), ylims=(0, model.grid.Ly))

    ω_plot = contourf(xω, yω, clamp.(ω_snapshot', -ω_lim, ω_lim);
                       color = :balance,
                      levels = ω_levels,
                       clims = (-ω_lim, ω_lim),
                      kwargs...)

    s_plot = contourf(xs, ys, clamp.(s_snapshot', 0, s_lim);
                       color = :thermal,
                      levels = s_levels,
                       clims = (0., s_lim),
                      kwargs...)

    plot(ω_plot, s_plot, title=["Vorticity" "Speed"], layout=(1, 2), size=(1200, 500))
end

┌ Info: Making a neat movie of vorticity and speed...
└ @ Main In[13]:3
┌ Info: Plotting frame 1 from iteration 0...
└ @ Main In[13]:7
┌ Info: Plotting frame 2 from iteration 101...
└ @ Main In[13]:7
┌ Info: Plotting frame 3 from iteration 201...
└ @ Main In[13]:7
┌ Info: Plotting frame 4 from iteration 302...
└ @ Main In[13]:7
┌ Info: Plotting frame 5 from iteration 403...
└ @ Main In[13]:7
┌ Info: Plotting frame 6 from iteration 504...
└ @ Main In[13]:7
┌ Info: Plotting frame 7 from iteration 605...
└ @ Main In[13]:7
┌ Info: Plotting frame 8 from iteration 706...
└ @ Main In[13]:7
┌ Info: Plotting frame 9 from iteration 807...
└ @ Main In[13]:7
┌ Info: Plotting frame 10 from iteration 908...
└ @ Main In[13]:7
┌ Info: Plotting frame 11 from iteration 1009...
└ @ Main In[13]:7
┌ Info: Plotting frame 12 from iteration 1110...
└ @ Main In[13]:7
┌ Info: Plotting frame 13 from iteration 1211...
└ @ Main In[13]:7
┌ Info: Plotting frame 14 from iteration 1312...
└ @ Main In[13]:7
┌ Info: Plo

Animation("C:\\Users\\shiva\\AppData\\Local\\Temp\\jl_MFyKFz", ["000001.png", "000002.png", "000003.png", "000004.png", "000005.png", "000006.png", "000007.png", "000008.png", "000009.png", "000010.png"  …  "000017.png", "000018.png", "000019.png", "000020.png", "000021.png", "000022.png", "000023.png", "000024.png", "000025.png", "000026.png"])

In [3]:
gif(anim, "anim_2_mue=1fps15.gif", fps = 2)

LoadError: UndefVarError: gif not defined